# Text and tables extraction pipeline

This notebook briefly presents how to run our extraction pipeline (in both single and batch mode) to extract and work with papers' text and tables.

## Setup

As specified in README, we have to create conda environment:

```conda env create -f environment.yml```

Additionally, we can pull in advance the required docker images:

```make pull_images```

Once the environment is activated (`conda activate xtables`) we can test the extraction pipeline with `make test`. To extract data in batch mode, run

```make DATA_DIR=mydata -j 8 -i extract_all > stdout.log 2> stderr.log```

where `8` is a number of processes to use. Under `mydata` there should be a directory named `arxiv/sources` containing  packed LaTeX sources, one archive per paper. After running the test above you can see the directory structure by inspecting `test/data/arxiv` directory.

Once the extraction is finished we can load all the papers. In the example below we load the paper from `test` directory.

In [1]:
from sota_extractor2.data.paper_collection import PaperCollection
pc = PaperCollection.from_files('test/data/arxiv', load_annotations=False)

`PaperCollection` is a list of papers. In the test example there's only one paper with two tables:

In [2]:
print(pc[0].text.title)
print(pc[0].text.abstract)

DILBERT: Distilling Inner Latent BERT variables
In this paper we achieve state-of-the-art performance in random number generation.


In [3]:
pc[0].tables[0]

left,center,right
1,2,3
4,5,6
7,8,9
a,b,c


In [4]:
pc[0].tables[1]

bold text,italic text,bold italic text
red text,green text,blue text
5.4%,3.8%,11.2±0.15
an italic text inside bold,bold red,


## Manual extraction

We can access the extraction pipeline programatically. In case of batch mode presented above we don't provide downloading of arXiv papers on purpose as for batch mode one should use arXiv's S3 bucket. In the example below we show how to load a paper directly from arXiv url, but use of it should be limited.

In [5]:
from sota_extractor2.data.paper_collection import PaperCollection, TempPaper
from pathlib import Path
from tempfile import TemporaryDirectory
from sota_extractor2.helpers import LatexConverter, Unpack
from sota_extractor2.errors import *
from urllib.request import urlretrieve as download_url

unpack = Unpack()
latex = LatexConverter(Path().absolute())

In [6]:
import re
arxiv_url_re = re.compile(r"^(https?://(www.)?arxiv.org/(abs|pdf|e-print)/)?(?P<arxiv_id>\d{4}\.\d+(v\d+)?)(\.pdf)?$")

def get_paper_html(path):
    """Get an HTML source of a given paper identified by either local path or arXiv url.
    
    :param path - either a local path to an archive with LaTeX source or arXiv url / id.
    In the later case the LaTeX source is downloaded automatically.
    
    :returns html - a string containing paper's HTML source code.
    """
    with TemporaryDirectory() as workdir:
        workdir = Path(workdir)
        m = arxiv_url_re.match(path)
        if m:
            path = workdir / "source"
            workdir = workdir / "unpack"
            download_url(f"https://arxiv.org/e-print/{m['arxiv_id']}", path)                
        unpack(path, workdir)
        html = latex.to_html(workdir)
        # Skipped step - reference normalization
    return html

def pipeline(path):
    """Run the whole pipeline, from path to proposals

    :param path - either a local path to an archive with LaTeX source or arXiv url / id.
    
    :returns paper - Paper object with labelled tables
    :returns proposals - DataFrame containing extracted sota records
    """
    try:
        html = get_paper_html(path)
        paper = TempPaper(html)
        return paper
    except UnpackError:
        print("Unable to unpack sources")
        return None
    except LatexConversionError:
        print("Unable to convert to html")
        return None

The extraction may take up to a few minutes, as it needs to:
* download the LaTeX source
* unpack them
* convert LaTeX to html
* clean the html
* extract paper's text, title, abstract, sections and references
* extract tables

In [7]:
paper = pipeline("https://arxiv.org/abs/1903.08469")

/home/ubuntu/anaconda3/envs/xxt/lib/python3.7/site-packages/bs4/__init__.py:177: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [8]:
paper.text.title

'In Defense of Pre-trained ImageNet Architecturesfor Real-time Semantic Segmentation of Road-driving Images'

In [9]:
paper.tables[0]

model,subset,mIoU,FPS,FPS norm,GPU,resolution,GFLOPs,GFLOPs@1Mpx,# params
D* [35],val,68.4,-,-,TitanX M,1024x512,5.8,11.6,0.5M
DG2s [35],val,70.6,-,-,TitanX M,1024x512,19.0,38,1.2M
Ladder DenseNet†[16],val,72.8,31.0,30.1,TitanX,1024x512,-,-,9.8M
ICNet [40],test,69.5,30.3,49.7,TitanX M,2048x1024,-,-,-
ESPNet [24],test,60.3,112,108.7,TitanX,1024x512,-,-,0.4M
ERFNet [28],test,68.0,11.2,18.4,TitanX M,1024x512,27.7,55.4,20M
GUNet†[23],test,70.4,37.3,33.3,TitanXP,1024x512,-,-,-
ERFNet†[28],test,69.7,11.2,18.4,TitanX M,1024x512,27.7,55.4,20M
SwiftNetRN-18,val,70.4,39.9,39.3,GTX 1080Ti,2048x1024,104.0,52.0,11.8M
SwiftNetMN V2,val,69.4,27.7,27.7,GTX 1080Ti,2048x1024,41.0,20.5,2.4M


Internally the cells are stored as a pandas dataframe of `Cell`s:

In [10]:
paper.tables[0].df.iloc[0,0]

Cell(value='model', raw_value='model', gold_tags='', refs=[], layout='border-r align-left header')

In [11]:
paper.tables[0].df.applymap(lambda cell: cell.value)

,0,1,2,3,4,5,6,7,8,9
0,model,subset,mIoU,FPS,FPS norm,GPU,resolution,GFLOPs,GFLOPs@1Mpx,# params
1,D* [35],val,68.4,-,-,TitanX M,1024x512,5.8,11.6,0.5M
2,DG2s [35],val,70.6,-,-,TitanX M,1024x512,19.0,38,1.2M
3,Ladder DenseNet†[16],val,72.8,31.0,30.1,TitanX,1024x512,-,-,9.8M
4,ICNet [40],test,69.5,30.3,49.7,TitanX M,2048x1024,-,-,-
5,ESPNet [24],test,60.3,112,108.7,TitanX,1024x512,-,-,0.4M
6,ERFNet [28],test,68.0,11.2,18.4,TitanX M,1024x512,27.7,55.4,20M
7,GUNet†[23],test,70.4,37.3,33.3,TitanXP,1024x512,-,-,-
8,ERFNet†[28],test,69.7,11.2,18.4,TitanX M,1024x512,27.7,55.4,20M
9,SwiftNetRN-18,val,70.4,39.9,39.3,GTX 1080Ti,2048x1024,104.0,52.0,11.8M


The paper's text can be accessed as well:

In [12]:
paper.text.fragments.print()

# xxanchor-S1 1 Introduction

xxanchor-S0F1 Figure 1: Speed-accuracy trade-off for different semantic semantic segmentation approaches on Cityscapes test on GTX1080Ti (except for DG2s which reports only validation performance). Red dots represent our method. Other methods are displayed in green, whereas blue dots show estimated frame rates of the corresponding methods on our GPU (please refer to subsection xxref-S4SS2 for details). Our submissions achieve the best accuracy and the best speed among all approaches aiming at real-time operation. Semantic segmentation is an important dense prediction task in which the inference targets posterior distribution over a known set of classes in each image pixel [xxref-bibbib6, xxref-bibbib20, xxref-bibbib3]. Currently, the best results are achieved with deep fully convolutional models which require extraordinary computational resources. Many important applications such as autonomous navigation or driver assistance require inference on very large images in order to cover a wide field of view and perceive pedestrians at distances of over 200m. At the same time, these applications require a very low latency in order to be able to bring real-time decisions. The resulting requirements intensify computational strain and make real-time implementations a challenging research objective.

1000

Many real-time semantic segmentation approaches [xxref-bibbib28, xxref-bibbib40, xxref-bibbib24, xxref-bibbib32] address this goal by introducing custom lightweight architectures which are not suited for large-scale visual recognition. Most of these approaches initialize training from scratch, and thus miss a huge regularization opportunity offered by knowledge transfer [xxref-bibbib26] from larger and more diverse recognition datasets [xxref-bibbib30]. Consequently, these methods incur a comparatively large overfitting risk. Some approaches alleviate this shortcoming by pre-training on ImageNet [xxref-bibbib28]. However, our experiments suggest that the resulting benefits tend to be smaller than in architectures which are designed for competitive ImageNet performance.

1001

...

19039

xxanchor-bibbib41 [41] H. Zhao, J. Shi, X. Qi, X. Wang, and J. Jia. Pyramid scene parsing network. In ICCV, 2017.

19040

xxanchor-bibbib42 [42] B. Zoph, V. Vasudevan, J. Shlens, and Q. V. Le. Learning transferable architectures for scalable image recognition. In The IEEE Conference on Computer Vision and Pattern Recognition (CVPR), June 2018.

19041

In [13]:
for header in paper.text.fragments.get_toc():
    print(header)

xxanchor-S1 1 Introduction
xxanchor-S2 2 Related Work
xxanchor-S3 3 The proposed segmentation method
xxanchor-S3SS1 3.1 Basic building blocks
xxanchor-S3SS2 3.2 Single scale model
xxanchor-S3SS3 3.3 Interleaved pyramid fusion model
xxanchor-S4 4 Experiments
xxanchor-S4SS1 4.1 Training details
xxanchor-S4SS2 4.2 Measuring the computational complexity
xxanchor-S4SS3 4.3 Cityscapes
xxanchor-S4SS4 4.4 CamVid
xxanchor-S4SS5 4.5 Validation of the upsampling capacity
xxanchor-S4SS6 4.6 Ablation of lateral connections
xxanchor-S4SS7 4.7 Execution profile
xxanchor-S4SS8 4.8 Size of the receptive field
xxanchor-S4SS9 4.9 Processing speed on Jetson TX2
xxanchor-S5 5 Conclusion
xxanchor-Sx1 Acknowledgment
xxanchor-bib References
